In [1]:
!wget https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/dev_aspects.txt

!wget https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/dev_reviews.txt

!wget https://raw.githubusercontent.com/dkulagin/kartaslov/master/dataset/kartaslovsent/kartaslovsent.csv

--2022-12-29 05:49:38--  https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/dev_aspects.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 57508 (56K) [text/plain]
Saving to: ‘dev_aspects.txt’

dev_aspects.txt     100%[===================>]  56.16K  --.-KB/s    in 0.003s  

2022-12-29 05:49:38 (17.3 MB/s) - ‘dev_aspects.txt’ saved [57508/57508]

--2022-12-29 05:49:39--  https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/dev_reviews.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK

In [ ]:
!pip install spacy==3.2.0
!python -m spacy download ru_core_news_lg

In [22]:
import pandas as pd
import tqdm
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import spacy

nlp = spacy.load('ru_core_news_lg')

# Скачаем тональный словарь русского языка [Картаслов](https://github.com/dkulagin/kartaslov/tree/master/dataset/kartaslovsent)

In [5]:
sent_dict = pd.read_csv('kartaslovsent.csv', sep=';')

In [6]:
sent_dict

,term,tag,value,pstv,ngtv,neut,dunno,pstvNgtvDisagreementRatio
0,абажур,NEUT,0.08,0.185,0.037,0.580,0.198,0.00
1,аббатство,NEUT,0.10,0.192,0.038,0.578,0.192,0.00
2,аббревиатура,NEUT,0.08,0.196,0.000,0.630,0.174,0.00
3,абзац,NEUT,0.00,0.137,0.000,0.706,0.157,0.00
4,абиссинец,NEUT,0.28,0.151,0.113,0.245,0.491,0.19
...,...,...,...,...,...,...,...,...
46122,ёмкость,NEUT,0.00,0.167,0.000,0.690,0.143,0.00
46123,ёрзать,NGTV,-0.54,0.050,0.446,0.397,0.107,0.00
46124,ёрничать,NGTV,-0.79,0.078,0.529,0.236,0.157,0.00
46125,ёрш,NEUT,0.16,0.224,0.072,0.576,0.128,0.00


Чтобы посмотреть, стоит ли заниматься даннным методом, проверим точность на золотом стандарте. В целом, кажется, более логичным использовать данный метод уже на выделенных аспектах. 

#### Можно попробовать оценить аспекты, соотнося их со словами из словаря и опираясь на их оценки.  
В словаре используются только 3 метки: положительное, отрицательное, нейтральное. Однако, в нашей классификации их 4: neutral, positive, both, negative. Предлагаю в таком случае смотреть на доли голосов при разметке, их как раз 4: pstv, ngtv, neut и dunno (перевес последнего можно относить к both)

In [7]:
gold = pd.read_csv('dev_aspects.txt', sep='\t', names=['id', 'category', 'aspect', 'start', 'end', 'sentiment'])
reviews = pd.read_csv('dev_reviews.txt', sep='\t', names=['id', 'text'])

In [8]:
reviews

,id,text
0,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ..."
1,1427,Здравствуйте!Посетили ваше заведение вчера пер...
2,16714,"Были в пятницу (19.03.10), заказывали столик д..."
3,797,"Были в ресторане 2 раза. Один раз днем, все по..."
4,34710,Удивляюсь отзывам про хорошее обслуживание. Бы...
...,...,...
66,9216,Вы брали этот ресторан так как он близко от до...
67,8996,"Были с друзьями в пабе Метрополь, всё очень по..."
68,38299,"Случайно увидели акцию на сайте купонов, решил..."
69,37819,Очень долго выбирали ресторан на Новогодний ка...


In [9]:
gold

,id,category,aspect,start,end,sentiment
0,3976,Whole,ресторане,71,80,neutral
1,3976,Whole,ресторанах,198,208,neutral
2,3976,Whole,ресторане,256,265,neutral
3,3976,Service,Столик бронировали,267,285,neutral
4,3976,Service,администратор,322,335,positive
...,...,...,...,...,...,...
1185,33043,Service,заказ,792,797,positive
1186,33043,Service,принесли,798,806,positive
1187,33043,Food,приготовили,880,891,positive
1188,33043,Service,оставил,1017,1024,negative


Лемматизируем аспекты

In [10]:
gold['lemmas'] = gold.apply(lambda x: " ".join([token.lemma_ for token in nlp(x['aspect'])]), axis = 1)

In [19]:
pred = []
for aspect in gold['lemmas']:
  score_dict = {'positive': 0, 'negative': 0, 'neutral': 0, 'both': 0}
  for lemma in aspect.split(' '):
    try:
      scores = sent_dict[sent_dict['term'] == lemma][['pstv', 'ngtv', 'neut', 'dunno']]
      score_dict['positive'] += float(scores['pstv'])
      score_dict['negative'] += float(scores['ngtv'])
      score_dict['neutral'] += float(scores['neut'])
      score_dict['both'] += float(scores['dunno'])
    except:
      continue
  pred.append(max(score_dict, key=score_dict.get))

In [20]:
gold['pred'] = pred

In [21]:
print("Accuracy: %.4f" % accuracy_score(list(gold['pred']), list(gold['sentiment'])))

Accuracy: 0.2908


# Попробуем еще раз. 
Будем смотреть на то, за какую тональность больше голосов, а также сравнивать голоса за негативную и положительную оценку: если разница будет меньше 0.05, то есть одного процента, а нейтральная оценка будет не больше 50%, то будет присваивать "both"

In [17]:
pred = []
for aspect in gold['lemmas']:
  score_dict = {'positive': 0, 'negative': 0, 'neutral': 0, 'both': 0}
  len_aspect = len(aspect.split(' '))
  for lemma in aspect.split(' '):
    try:
      scores = sent_dict[sent_dict['term'] == lemma][['pstv', 'ngtv', 'neut', 'dunno']]
      score_dict['positive'] += float(scores['pstv'])
      score_dict['negative'] += float(scores['ngtv'])
      score_dict['neutral'] += float(scores['neut'])
      score_dict['both'] += float(scores['dunno'])
    except:
      continue
  score_dict['positive'] = score_dict['positive'] / len_aspect
  score_dict['negative'] = score_dict['negative'] / len_aspect
  score_dict['neutral'] = score_dict['neutral'] / len_aspect
  score_dict['both'] = score_dict['both'] / len_aspect
  
  if abs(score_dict['positive'] - score_dict['negative']) < 0.05 and score_dict['both'] < 0.5:
    pred.append('both')
  else:
    pred.append(max(score_dict, key=score_dict.get))

In [18]:
print("Accuracy: %.4f" % accuracy_score(pred, list(gold['sentiment'])))

Accuracy: 0.2420


Стало хуже... 

Давайте посмотрим на первое слово

In [31]:
pred = []
for aspect in gold['lemmas']:
  score_dict = {'positive': 0, 'negative': 0, 'neutral': 0, 'both': 0}
  len_aspect = len(aspect.split(' '))
  lemma = aspect.split(' ')[0]
  try:
    scores = sent_dict[sent_dict['term'] == lemma][['pstv', 'ngtv', 'neut', 'dunno']]
    score_dict['positive'] += float(scores['pstv'])
    score_dict['negative'] += float(scores['ngtv'])
    score_dict['neutral'] += float(scores['neut'])
    score_dict['both'] += float(scores['dunno'])
    if abs(score_dict['positive'] - score_dict['negative']) < 0.05 and score_dict['both'] < 0.5:
      pred.append('both')
    else:
      pred.append(max(score_dict, key=score_dict.get))
  except:
    pred.append('neut')  

In [32]:
print("Accuracy: %.4f" % accuracy_score(pred, list(gold['sentiment'])))

Accuracy: 0.2387


## Проблемы метода. 
- в словаре нет необходимой нам разметки "both" или ей подобной, поэтому разметка аспектов осложняется
- если смотреть на value слова в словаре, то мы никак не будем различать "neutral" и "both"
- словарь не контекстно зависимый, поэтому многие оценки очевидно не соотвествуют нашим данным. Так, например, прилагательное "высокий" имеет нейтральную оценку, а количество голосов за положительную оценку опережает количество за отрицательную. Однако, в нашем случае "высокая цена" будет иметь скорее отрицательную коннатацию, а "высокий уровень обслуживания" - положительную. 
